In [4]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [5]:
price_data = pd.read_csv('stockPriceData-2.csv')
price_data_500 = price_data.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df = price_data[price_data['Symbol'].isin(price_data_500['Symbol'])]
df['Date'] = pd.to_datetime(df['Date'])
master_date = df.drop_duplicates(subset='Date')[['Date']].reset_index(drop=True)
df.set_index('Date', inplace=True)

df = df.sort_values(by=['Date','Mcap'], ascending=[True,False])
df = df.sort_values(['Symbol', 'Date'])
df['PrevClose'] = df.groupby('Symbol')['Close'].shift(1)
df['returns'] = (df['Close'] - df['PrevClose']) / df['PrevClose']

In [6]:
# Function to calculate log returns
def calculate_log_returns(df):
    df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
    return df.dropna()

# Apply log return calculation
df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns)

# Function to calculate annualized standard deviation over a 252-day rolling window
def calculate_annualized_std(df):
    return df['LogReturn'].rolling(window=252).std() * np.sqrt(252)

# Function to calculate downside volatility (only negative returns)
def calculate_downside_vol(df):
    neg_returns = df['LogReturn'].copy()
    neg_returns[neg_returns > 0] = 0  # Keep only non-positive returns
    return neg_returns.rolling(window=252).std() * np.sqrt(252)

# Calculate metrics
df['LowVol'] = df.groupby('Symbol', group_keys=False).apply(calculate_annualized_std)
df['DownsideVol'] = df.groupby('Symbol', group_keys=False).apply(calculate_downside_vol)

# Reset index if needed
df =df[['Symbol','LowVol','DownsideVol','Mcap']]
df = df.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df

,Symbol,LowVol,DownsideVol,Mcap
Date,,,,
1995-06-19,RELIANCE,NaN,NaN,125220.586144
1995-06-19,TATASTEEL,NaN,NaN,79968.652924
1995-06-19,HINDPETRO,NaN,NaN,67989.600000
1995-06-19,GRASIM,NaN,NaN,47112.551455
1995-06-19,NESTLEIND,NaN,NaN,20375.854648
...,...,...,...,...
2025-06-13,NSLNISP,0.349457,0.206406,114440.158443
2025-06-13,GENUSPOWER,0.492984,0.292618,112605.359198
2025-06-13,WESTLIFE,0.357138,0.192084,111658.090948


In [7]:
df = df[['Symbol','LowVol','DownsideVol']]
# First, identify the columns to rank (excluding 'Date' and 'Symbol')
rank_cols = [col for col in df.columns if col not in ['Symbol']]

# Apply cross-sectional percentile rank for each date
df[rank_cols] = df.groupby('Date')[rank_cols].transform(lambda x: x.rank(pct=True, ascending=False))
df

,Symbol,LowVol,DownsideVol
Date,,,
1995-06-19,RELIANCE,NaN,NaN
1995-06-19,TATASTEEL,NaN,NaN
1995-06-19,HINDPETRO,NaN,NaN
1995-06-19,GRASIM,NaN,NaN
1995-06-19,NESTLEIND,NaN,NaN
...,...,...,...
2025-06-13,NSLNISP,0.589852,0.526427
2025-06-13,GENUSPOWER,0.145877,0.095137
2025-06-13,WESTLIFE,0.547569,0.617336


In [8]:
df.to_csv('lv+dv.csv')